## First rough attempt at topographic analysis in Python
#### slower and clunkier than I'd like but potentially promising

In [ ]:
import rasterio
import numpy as np

In [2]:
# Function to load DEM data from a .tif file
def load_dem(filename):
    with rasterio.open(filename) as dem_file:
        dem_data = dem_file.read(1)  # Read the first (and typically only) band of data
    return dem_data

In [3]:
# Load and visualize the DEM
dem_filename = 'MOLA-HRSC Blended/Mars_HRSC_MOLA_BlendDEM_Global_200mp_v2.tif' # this is a really big file and takes forever to run
dem_data = load_dem(dem_filename)

In [4]:
def calculate_flow_directions(dem_path):
    dem = load_dem(dem_path)

    # Initialize the flow direction array with zeros
    flow_directions = np.zeros_like(dem, dtype=np.uint8)

    # Define the function to find the lowest neighbor
    def get_lowest_neighbor(x, y, grid):
        # Define possible directions and their corresponding outputs
        directions = {
            (0, -1): 1,  # North
            (1, -1): 2,  # Northeast
            (1, 0): 3,   # East
            (1, 1): 4,   # Southeast
            (0, 1): 5,   # South
            (-1, 1): 6,  # Southwest
            (-1, 0): 7,  # West
            (-1, -1): 8, # Northwest
        }

        min_elevation = grid[x, y]
        flow_dir = 0  # No flow by default

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < grid.shape[0] and 0 <= ny < grid.shape[1]:
                if grid[nx, ny] < min_elevation:
                    min_elevation = grid[nx, ny]
                    flow_dir = directions[(dx, dy)]

        return flow_dir

    # Iterate over each cell
    for i in range(dem.shape[0]):
        for j in range(dem.shape[1]):
            flow_directions[i, j] = get_lowest_neighbor(i, j, dem)

    # Write the flow direction raster
    output_path = dem_path.replace('.tif', '_flow_directions.tif')
    with rasterio.open(output_path, 'w', driver='GTiff', height=dem.shape[0], width=dem.shape[1], count=1, dtype=flow_directions.dtype, crs='+proj=latlong') as out_dataset:
        out_dataset.write(flow_directions, 1)

    return output_path

In [ ]:
# i have a big file pre-loaded so it takes a really long time, feel free to use a smaller DEM
flow_directions_path = calculate_flow_directions(dem_filename)